In [1]:
#imports

import matplotlib.pyplot as plt
import pandas as pd,json
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin
from datetime import date, datetime, timedelta
from sklearn.pipeline import Pipeline
import branca.colormap as cm
import ipyleaflet as lf
from ipyleaflet import LayerGroup,WidgetControl
import ipywidgets as widgets
from ipyleaflet import basemaps, interactive
from math import sin, cos, sqrt, atan2,radians

In [2]:
###Functions from Dr. S's code

def parseTStamp(tstamp_df):    
    return datetime.strptime(tstamp_df['timeAtPosition'], '%Y-%m-%dT%H:%M:%S%z')


def dms2dec(dms_tuple):
   # print(dms_tuple)
    
    deg, minutes, seconds, direction = dms_tuple
    if np.isnan(seconds):
        seconds=0
    # print("Out", deg, minutes, seconds, direction)
    sign = 1
    direction = direction.strip()
    if direction == 'W' or direction == 'WEST' or direction =='S' or direction=='SOUTH':
        sign=-1
    return sign*(deg + minutes/60.0 + seconds/(60.0*60.0))


def simple2alt(alt_df):
    try:
        return int(alt_df['simpleAltitude'].strip('C').strip('T')) *100
    except Exception as exp:
        return alt_df['simpleAltitude']
    
    
class TFMSDataPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
    
    
    def fit(self, X, y=None):
        return self
    
    
    def transform(self, X):        
        X['lat'] = X[['latitudeDMS_degrees','latitudeDMS_minutes','latitudeDMS_seconds', 'latitudeDMS_direction']].apply(dms2dec, axis=1)
        X['long'] = X[['longitudeDMS_degrees','longitudeDMS_minutes','longitudeDMS_seconds', 'longitudeDMS_direction']].apply(dms2dec, axis=1)
        X['alt'] = X[['simpleAltitude']].apply(simple2alt, axis=1)
        X['tstamp'] = X[['timeAtPosition']].apply(parseTStamp, axis=1)
        return X[self.attributes]
    

extract_attributes = [
    'tstamp',
    'flightRef',
    'acid',
    'gufi',    
    'lat',
    'long',
    'alt',
    'speed',
    'depArpt',
    'arrArpt']

In [3]:
#create a dictionary of data with keys=csv file names and corresponding values are dataframes
def getDataDict(listOfCsvFiles):
    data_dict={}
    for csv in listOfCsvFiles:
        data_dict[csv]=pd.read_csv(csv)
    return data_dict

#clean the data and extract data of interest
def cleanData(data_dict):
    wpts_data_dict={}
    pipe = Pipeline([('SELECT', TFMSDataPreprocessor(extract_attributes)), ])
    for key in data_dict:
        wpts_data_dict[key]=pipe.fit_transform(data_dict[key])
        wpts_data_dict[key]=wpts_data_dict[key].dropna()   
    return wpts_data_dict

#create lists of lats,longs and altitudes
def generateLists(wpts_data_dict):
    lats=[]
    longs=[]
    alts=[]
    
    for key in wpts_data_dict:
        lats.append(wpts_data_dict[key]['lat'].tolist())
        longs.append(wpts_data_dict[key]['long'].tolist())
        alts.append(wpts_data_dict[key]['alt'].tolist())
    return lats,longs,alts

#get absolute min and max alt values across the files
def getAbsoluteMinMax(listOfAlts):
    abmin=0
    abmax=0
    min_alt=[]
    max_alt=[]
    for i in range(0,len(listOfAlts)):
        min_alt.append(min(listOfAlts[i]))
        max_alt.append(max(listOfAlts[i]))
    abmin=min(min_alt)
    abmax=max(max_alt)
    return abmin,abmax


#add an altitude slider to the map
def AltSlider(absmin_alt,absmax_alt):
    return widgets.FloatRangeSlider(value=[absmin_alt,absmax_alt],min=absmin_alt,max=absmax_alt,step=1000,description='altitude:',
                                   disabled=False,continuous_update=False,orientation='horizontal',
                                   readout=True,readout_format='1d',layout=widgets.Layout(width='350px'))



#get a subset of data based on the min and max values selected on the altitude slider
def dataSubsetForPlotting(wpts_data_dict,low,high):
    wpts_data_dict=wpts_data_dict
    requiredData={}
    for key in wpts_data_dict:
        df=wpts_data_dict[key]
        df=df[df['alt'].between(low,high)]
        requiredData[key]=df
    return requiredData

lg1 = None
lg2 = None

#function to put data as markers on the map
def plotData(data,listoffiles,m):
        
    lats,longs,altitudes=generateLists(data)
    try:
        global lg1
        global lg2
        
         
        if lg1 is not None:
            m.remove_layer(lg1)
        if lg2 is not None:
            m.remove_layer(lg2)
    except:
        pass
    lg1=LayerGroup()
    lg2=LayerGroup()
    for key in data:
        print(key)
        if key==listoffiles[0]:
            lg1.name=key
            lg1.layers=[lf.CircleMarker(location=(lats[0][ii],longs[0][ii]),
                                            radius=3,
                                            stroke=False,
                                            fill=True,
                                            fill_color=cmap(altitudes[0][ii]),
                                            weight=0,
                                            fill_opacity=1) for ii in range(0,len(altitudes[0]))]
            m.add_layer(lg1)
            #return lg1

        if key==listoffiles[1]:
            rect_list=[]
            for ii in range(0,len(altitudes[1])):
                pt=(lats[1][ii],longs[1][ii])
                W=pt[1]-0.0001
                E=pt[1]+0.0001
                N=pt[0]+0.0001
                S=pt[0]-0.0001

                upper_left=(N,W)
                upper_right=(N,E)
                lower_right=(S,E)
                lower_left=(S,W)

                edges = [upper_left, upper_right, lower_right, lower_left]
                rect_list.append(lf.Rectangle(bounds=(edges[2:],edges[:2]),
                                          fill=True,
                                          color=cmap(altitudes[1][ii]),
                                          weight=3,
                                          fill_opacity=1))

            lg2.name=key
            lg2.layers=rect_list
            m.add_layer(lg2)
            #m.add_layer(lg2)
        #return lg1,lg2
    

class dataplotter():
    #lg=LayerGroup()
    
    def buildmap(self):
        return lf.Map(center=[32.8917,-97.0417], zoom_start=9,basemap=basemaps.Stamen.Terrain)
    
#function to calculate one point's distance from the center which are the coordinaes of DFW    
def calculateDistance(lat,lon,centerlat,centerlon):
    R = 6373.0
    lat = radians(lat)
    lon = radians(lon)
    centerlat = radians(centerlat)
    centerlon = radians(centerlon)

    dlon = centerlon - lon
    dlat = centerlat - lat

    a = sin(dlat / 2)**2 + cos(lat) * cos(centerlat) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

#function to return a dataframe including values only withing 10 nautical miles distance from DFW airport
def dataSubsetForBuffer(X):
    X['dist']=list(map(lambda k: calculateDistance(X.loc[k]['lat'],X.loc[k]['long'],32.8917,-97.0417), X.index))
    return X[X['dist']<18.52]

In [4]:
tfms_csv_files = ['KDFW20190512_170000.csv','KDFW20190512_160000.csv']
#tfms_csv_files = ['KDFW20190512_170000.csv']

In [5]:
tfms_data_dict=getDataDict(tfms_csv_files)
flights_wpts_dict=cleanData(tfms_data_dict)

testSmalldf={}  ##get a test subset of data withing 10 nautical miles

for key in flights_wpts_dict:
    #print(flights_wpts_dict[key].shape)
    testSmalldf[key]=dataSubsetForBuffer(flights_wpts_dict[key])
    #print(testSmalldf[key].shape)

#print(testSmalldf.values())

lats1,longs1,alts1=generateLists(testSmalldf)
vmin,vmax=getAbsoluteMinMax(alts1)



#build map
cp=dataplotter()
mymap = cp.buildmap()

colors=['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
cmap=cm.LinearColormap(colors=colors, vmin=vmin, vmax=vmax)   
cmap.caption='Flight altitude'

displayplots=LayerGroup()


#for slider
altitudeSlider=AltSlider(vmin,vmax)

loweralt=altitudeSlider.value[0]
higheralt=altitudeSlider.value[1]

altOutput=widgets.Output()

#datatobeplotted=None
#display(altitudeSlider,altOutput)
print("Before")
def onAltValueChange(change):
    with altOutput:
                ###a LayerGroup() to clear the previous layes on the map
        
        global datatobeused
        
        print("Here")

        loweralt=altitudeSlider.value[0]
        higheralt=altitudeSlider.value[1]
        
        datatobeused=dataSubsetForPlotting(testSmalldf,loweralt,higheralt)
        
        plotData(datatobeused,tfms_csv_files,mymap)
        #mymap.add_layer(lg1)
        #mymap.add_layer(lg2)

altitudeSlider.observe(onAltValueChange,names='value')
'''
def f(loweralt, higheralt):
    global datatobeused
    datatobeused=dataSubsetForPlotting(testSmalldf,loweralt,higheralt)
    plotData(datatobeused,tfms_csv_files,mymap)

interactive_plot = interactive(f, loweralt=(vmin, vmax), higheralt=(vmin,vmax))
'''
#add colormap to the map
out = widgets.Output(layout={'border': '1px solid black',})
with out:
    display(cmap)
cmap_control = WidgetControl(widget=out, position='bottomright')
mymap.add_control(cmap_control)


mymapWidgetControl = WidgetControl(widget = altitudeSlider, position = 'bottomleft')
mymap.add_control(mymapWidgetControl)
mymap.add_control(lf.LayersControl(position='topright'))
mymap.add_control(lf.FullScreenControl())


#f(vmin, vmax)

#widgets.interact(printValues,rangefromslider=altitudeSlider)

Before


In [6]:
mymap.save('mylfwidgetsmap.html')

In [7]:
mymap

Map(center=[32.8917, -97.0417], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…